# Import earth engine api, geemap and authenticate GEE

In [1]:
# Import earth engine python api and geemap 
import geemap, ee

# Authenticate the earthengine with credentials
ee.Initialize()

# Provide area of interest for calculation

In [3]:
# get our Nepal boundary
# I have taken level 0 data for country data from FAO datasets
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0") \
   .filter(ee.Filter.eq('ADM0_NAME','Nepal')).geometry() # adjust indentation here, May get error

# Define function for EVI calculations

In [11]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8').divide(10000),
            'RED': image.select('B4').divide(10000),
            'BLUE': image.select('B2').divide(10000)
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

### Date of image, addtion in image collection as a properties

In [6]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

# Filter image and apply EVI computation formula

In [17]:
# filter sentinel 2 images and apply the EVI formula, finally we obtain 
# single image with median operation

Sentinel_data = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate("2022-03-01","2022-03-31").filterBounds(aoi) \
    .map(getEVI)

# Computation of EVI 
### we will be using formula: LAI = (3.618*EVI - 0.118) for LAI

In [18]:
def getLAI(image):
    LAI = image.expression(
        '(3.618*EVI - 0.118)', {
            'EVI': image.select('EVI')
        }).rename("LAI")
    image = image.addBands(LAI)

    return(image)

# Apply LAI computation function to the image 

In [23]:
Lai_image = Sentinel_data.map(getLAI).map(addDate).median().clip(aoi)

# Add layers to map and visualize it.

In [27]:
# set some thresholds
#  A nice EVI palette
palett = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']

pall = {"min":0.5, "max":3.5, 'palette':palett}
map1 = geemap.Map()
map1.centerObject(aoi, 8)
map1.addLayer(Lai_image.select('LAI'), pall, "LAI")

map1.addLayerControl()
map1

Map(center=[28.268224502649357, 83.97637443188768], controls=(WidgetControl(options=['position', 'transparent_…